###    Exercising with titanic data  ---------------------------
####     'rowboat'! Get it?! You get it.

#### Exploration ---------------------------------------------------------------

In [14]:
import pandas as pd 

In [15]:
df_ti = pd.read_csv('train.csv')

In [16]:
df_ti.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [17]:
# count of survived & dead in training data - Survived:  0 = No, 1 = Yes

df_ti.groupby(by='Survived').Sex.count()  

Survived
0    549
1    342
Name: Sex, dtype: int64

In [18]:
# survival rate by sex 

df_ti.groupby(by='Sex').Survived.mean()

Sex
female    0.742038
male      0.188908
Name: Survived, dtype: float64

#### group by syntax -------------------------

In [19]:
df_ti.groupby(['Sex','Pclass']).Survived.mean()

Sex     Pclass
female  1         0.968085
        2         0.921053
        3         0.500000
male    1         0.368852
        2         0.157407
        3         0.135447
Name: Survived, dtype: float64

In [20]:
# Survival rate by sex & ticket class as a proxy for social class. 'Ladies first'

df_ti.groupby(['Sex','Pclass']).Survived.mean().unstack() # Pclass or ticket class: 1 = 1st, 2 = 2nd, 3 = 3rd

# same as: 
# df_ti.groupby(['Sex','Pclass']).Survived.aggregate('mean').unstack()

Pclass,1,2,3
Sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


In [21]:
# Avg age by sex & class

df_ti.groupby(['Sex','Pclass']).Age.mean().unstack()

Pclass,1,2,3
Sex,,,
female,34.611765,28.722973,21.750000
male,41.281386,30.740707,26.507589


In [22]:
# Avg age of *survivors* by sex & class

df_ti[df_ti.Survived==1].groupby(['Sex','Pclass']).Age.mean().unstack()

Pclass,1,2,3
Sex,,,
female,34.939024,28.080882,19.329787
male,36.248000,16.022000,22.274211


In [23]:
# Avg age of *non-survivors* by sex & class. 

df_ti[df_ti.Survived==0].groupby(['Sex','Pclass']).Age.mean().unstack()

Pclass,1,2,3
Sex,,,
female,25.666667,36.000000,23.818182
male,44.581967,33.369048,27.255814


In [24]:
# Avg 1st class female age (25.666667) is the only group w a younger avg age. Skewed by low sample size? 
# The answer is (drumroll please..): yes

df_ti[df_ti.Survived==0].groupby(['Sex','Pclass']).PassengerId.count().unstack()

Pclass,1,2,3
Sex,,,
female,3,6,72
male,77,91,300


In [25]:
# Easier to compare avg ages of survivors & dead. Survived:  0 = No, 1 = Yes

df_ti.groupby(['Sex','Pclass','Survived']).Age.mean().unstack()

Survived               0          1
Sex    Pclass                      
female 1       25.666667  34.939024
       2       36.000000  28.080882
       3       23.818182  19.329787
male   1       44.581967  36.248000
       2       33.369048  16.022000
       3       27.255814  22.274211

#### pivot table syntax -----------------------

In [26]:
# survival by class & sex

df_ti.pivot_table('Survived', index='Sex', columns='Pclass')

# vs. df_ti.groupby(['Sex','Pclass']).Survived.mean().unstack() 

Pclass,1,2,3
Sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


In [27]:
# class & sex

df_ti.pivot_table('Age',index='Sex',columns='Pclass')

# vs. df_ti.groupby(['Sex','Pclass']).Age.mean().unstack()

Pclass,1,2,3
Sex,,,
female,34.611765,28.722973,21.750000
male,41.281386,30.740707,26.507589


In [28]:
# group passengers by age group 0-18, 18-80

agers = pd.cut(df_ti.Age,[0,18,80])

df_ti.pivot_table('Survived',['Sex',agers],'Pclass')

Pclass                  1         2         3
Sex    Age                                   
female (0, 18]   0.909091  1.000000  0.511628
       (18, 80]  0.972973  0.900000  0.423729
male   (0, 18]   0.800000  0.600000  0.215686
       (18, 80]  0.375000  0.071429  0.133663

In [29]:
# pull out children (<5):  0-5, 5-18, 18-80

agers2 = pd.cut(df_ti.Age,[0,5,18,30,80])

df_ti.pivot_table('Survived',['Sex',agers2],'Pclass')

Pclass                  1         2         3
Sex    Age                                   
female (0, 5]    0.000000  1.000000  0.733333
       (5, 18]   1.000000  1.000000  0.392857
       (18, 30]  0.958333  0.900000  0.500000
       (30, 80]  0.980000  0.900000  0.304348
male   (0, 5]    1.000000  1.000000  0.384615
       (5, 18]   0.666667  0.142857  0.157895
       (18, 30]  0.428571  0.027027  0.147541
       (30, 80]  0.360000  0.106383  0.112500

In [30]:
# counts of passengers by age & sex

df_ti.groupby(['Sex',agers2]).PassengerId.count().unstack()

# df_ti.groupby(['Sex',agers2, 'Survived']).PassengerId.count().unstack()

# groupbys from above:
# df_ti.groupby(['Sex','Pclass']).Survived.mean()              # survival rate by sex & class
# df_ti.groupby(['Sex','Pclass']).Age.mean().unstack()         # above: avg age by sex & class

Age,"(0, 5]","(5, 18]","(18, 30]","(30, 80]"
Sex,,,,
female,21,47,90,103
male,23,48,180,202


In [31]:
df_ti.groupby(['Sex',agers2, 'Survived']).PassengerId.count().unstack()

Survived           0   1
Sex    Age              
female (0, 5]      5  16
       (5, 18]    17  30
       (18, 30]   22  68
       (30, 80]   20  83
male   (0, 5]      8  15
       (5, 18]    39   9
       (18, 30]  152  28
       (30, 80]  161  41

### ------------------------------------------------------------------

#### Data Dictionary

#### Variable	  Definition	                      Key
survival 	  Survival 	            0 = No, 1 = Yes
pclass   	  Ticket class 	        1 = 1st, 2 = 2nd, 3 = 3rd
sex 	      Sex 	
Age 	      Age in years 	
sibsp 	      # of siblings / spouses aboard the Titanic 	
parch 	      # of parents / children aboard the Titanic 	
ticket 	      Ticket number 	
fare 	      Passenger fare 	
cabin 	      Cabin number 	
embarked 	  Port of Embarkation 	C = Cherbourg, Q = Queenstown, S = Southampton

from https://www.kaggle.com/c/titanic/data